#### 自然言語処理 入門
# テキストデータの前処理

### 自然言語処理とは
人間が日常的に使っている「自然言語」をコンピュータに処理させる一連の技術のことです。

__自然言語処理における機械学習のフロー__
![](https://qiita-image-store.s3.amazonaws.com/0/77079/824d44eb-6499-add8-73ed-2aa923d02bc8.png)


自然言語処理をやろうと思うと、__前処理（Pre-processing）__が今まで以上に大切になります。

テキストの状態のデータは文字の羅列であり構造化されていません。それを今までのように数値データのベクトルに変換しなければなりません。
さらに、ノイズがあったり文字の長さが均一では無いので、上手に前処理をしないと精度が出ないことが多いです。

ですので今回は、__テキストデータの前処理__を主に勉強していきます。

## 前処理の主な流れ
![](https://s3.amazonaws.com/ai-standard/nlp-1-1.png)

## 単語の分割
自然言語処理では、__文章（テキスト）を単語レベルに区切って扱う__ことがほとんどです。ベクトル化する為には、テキストを分割するという前準備が必要になります。
<br>
![](https://s3.amazonaws.com/ai-standard/nlp-1-2.png)

ここで、日本語特有の問題が発生します。英語は、単語の区切りが空白で示されていますが、__日本語には区切りが無い__ということです。

日本語のように単語の区切りが明らかでない言語に対して行われるのが、単語の分割をする処理です。今回は__「形態素解析」__と__「N-gram」__という代表的な2つの手法を解説していきます。

### 形態素解析
#### 「形態素」とは？
文章を、それ以上分解したら意味をなさなくなる要素にまで分割したものです。

![](https://s3.amazonaws.com/ai-standard/nlp-1-3.png)

このように、形態素で分割したものを__わかち書き__と言います。

pythonには、形態素解析をしてくれる__MeCab__というパッケージがあるので、早速インストールしてみましょう。

インストールは以下のコマンド一つで完了します。（macの場合）

In [5]:
!pip install mecab-python3

では早速、MeCabを使ってみましょう。

In [11]:
import MeCab


mecab = MeCab.Tagger()
text = "私は東京都に住んでます"

# parse()の引数に文字列を入力すると解析結果が返ってきます。
print(mecab.parse(text))

私	名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
東京	名詞,固有名詞,地域,一般,*,*,東京,トウキョウ,トーキョー
都	名詞,接尾,地域,*,*,*,都,ト,ト
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
住ん	動詞,自立,*,*,五段・マ行,連用タ接続,住む,スン,スン
で	助詞,接続助詞,*,*,*,*,で,デ,デ
ます	助動詞,*,*,*,特殊・マス,基本形,ます,マス,マス
EOS



In [13]:
# Taggerの引数を"-Owakati"にすると、わかち書きされたテキストが返されます
mecab = MeCab.Tagger("-Owakati")
text = "私は東京都に住んでます"

print(mecab.parse(text))

私 は 東京 都 に 住ん で ます 



### N-gram
N文字単位で文字列を分解する処理を行います。
![](https://s3.amazonaws.com/ai-standard/nlp-1-4.png)

In [21]:
# 文字列を受け取り、指定されたN-gramのリストを返す関数

def ngram(words,N):
    ngram = []

    for i in range(len(words)):
        cw = ""
        if i >= N-1:
            for j in reversed(range(N)):
                cw += words[i-j]
        else:
            continue
        ngram.append(cw)

    return ngram

In [22]:
ngram("私は東京都に住んでいます", 1)

['私', 'は', '東', '京', '都', 'に', '住', 'ん', 'で', 'い', 'ま', 'す']

In [23]:
ngram("私は東京都に住んでいます", 2)

['私は', 'は東', '東京', '京都', '都に', 'に住', '住ん', 'んで', 'でい', 'いま', 'ます']

以上で、最初のステップである「単語の分割」が終わりました。

![](https://s3.amazonaws.com/ai-standard/nlp-1-1.png)

次からは、「単語の選択」を見ていきたいと思います。

## 単語の選択

### ステミング
主に英語などの欧米系の言語では、意味的には同じ単語が活用形などの影響で違う文字列になっていることがあります。
さらに、複数形や単数形といったような違いもしばしばあります。

ステミングは、単語からすべての接辞を取り除く処理を行います。

![](https://s3.amazonaws.com/ai-standard/nlp-1-5.png)

ステミングをするには、NLTKパッケージをインストールします。

In [25]:
!pip install -U nltk

Requirement already up-to-date: nltk in /Users/tsuruoka/.pyenv/versions/anaconda3-2.4.0/envs/jupyter/lib/python3.5/site-packages
Requirement already up-to-date: six in /Users/tsuruoka/.pyenv/versions/anaconda3-2.4.0/envs/jupyter/lib/python3.5/site-packages (from nltk)


In [32]:
import nltk

stemmer = nltk.PorterStemmer()

# programへステミング
print(stemmer.stem('programming'))
print(stemmer.stem('programs'))

# initiへステミング
print(stemmer.stem('initial'))
print(stemmer.stem('initialize'))
print(stemmer.stem('initialization'))

program
program
initi
initi
initi


### ストップワードの除去
トップワードというのは自然言語処理する際に一般的で役に立たない等の理由で処理対象外とする単語のことです。

たとえば、助詞や助動詞(「は」「の」「です」「ます」など)が挙げられます。これらの単語は出現頻度が高い割に有効な情報をもたらしません。さらに、計算量や性能に悪影響を及ぼすこともあるので除去するのが一般的です。

今回は以下の方式で、ストップワードの除去をしていきます。
- 辞書による方式
- 出現頻度による方式

#### 辞書によるストップワードの除去
あらかじめストップワードの辞書（リスト）を作成しておき、辞書内に含まれる単語をテキストから除去します。

その際に、既に公開されている日本語のストップワード辞書のひとつである、[Slothlib](http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt)を使うのがいいでしょう。

In [35]:
import urllib.request

slothlib_path = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
slothlib_file = urllib.request.urlopen(slothlib_path)
slothlib_stopwords = [line.decode("utf-8").strip() for line in slothlib_file]
slothlib_stopwords = [ss for ss in slothlib_stopwords if not ss==u'']

In [36]:
# ストップワードリスト

slothlib_stopwords

['あそこ',
 'あたり',
 'あちら',
 'あっち',
 'あと',
 'あな',
 'あなた',
 'あれ',
 'いくつ',
 'いつ',
 'いま',
 'いや',
 'いろいろ',
 'うち',
 'おおまか',
 'おまえ',
 'おれ',
 'がい',
 'かく',
 'かたち',
 'かやの',
 'から',
 'がら',
 'きた',
 'くせ',
 'ここ',
 'こっち',
 'こと',
 'ごと',
 'こちら',
 'ごっちゃ',
 'これ',
 'これら',
 'ごろ',
 'さまざま',
 'さらい',
 'さん',
 'しかた',
 'しよう',
 'すか',
 'ずつ',
 'すね',
 'すべて',
 'ぜんぶ',
 'そう',
 'そこ',
 'そちら',
 'そっち',
 'そで',
 'それ',
 'それぞれ',
 'それなり',
 'たくさん',
 'たち',
 'たび',
 'ため',
 'だめ',
 'ちゃ',
 'ちゃん',
 'てん',
 'とおり',
 'とき',
 'どこ',
 'どこか',
 'ところ',
 'どちら',
 'どっか',
 'どっち',
 'どれ',
 'なか',
 'なかば',
 'なに',
 'など',
 'なん',
 'はじめ',
 'はず',
 'はるか',
 'ひと',
 'ひとつ',
 'ふく',
 'ぶり',
 'べつ',
 'へん',
 'ぺん',
 'ほう',
 'ほか',
 'まさ',
 'まし',
 'まとも',
 'まま',
 'みたい',
 'みつ',
 'みなさん',
 'みんな',
 'もと',
 'もの',
 'もん',
 'やつ',
 'よう',
 'よそ',
 'わけ',
 'わたし',
 'ハイ',
 '上',
 '中',
 '下',
 '字',
 '年',
 '月',
 '日',
 '時',
 '分',
 '秒',
 '週',
 '火',
 '水',
 '木',
 '金',
 '土',
 '国',
 '都',
 '道',
 '府',
 '県',
 '市',
 '区',
 '町',
 '村',
 '各',
 '第',
 '方',
 '何',
 '的',
 '度',
 '文',
 '者',
 '性',
 '体',
 '人',
 '他',
 '今',
 

#### 出現頻度による除去
テキスト内の全ての単語出現頻度をカウントし、高頻度の単語をテキストから除去する方法です。以下の図は、ある英語の本の最も頻出する50単語の累計頻度をプロットしたものです。
![](https://s3.amazonaws.com/ai-standard/nlp-1-6.png)

50単語を見てみると、theやof、カンマのような文書分類等に役に立たなそうな単語がテキストの50%近くを占めていることが分かります。

以上で、「単語の選択」が終わりました。

![](https://s3.amazonaws.com/ai-standard/nlp-1-1.png)

最後に、「ベクトル化」について勉強していきましょう。

## ベクトル化
Bag-of-Words とは、文章に単語が含まれているかどうかのみを考え、単語の並び方などは考慮しないモデルのことです。一番シンプルなモデルは単語があれば 1、なければ 0 となります。

まずは、テキストをわかち書きして、出現した単語をリスト化します。

![](https://s3.amazonaws.com/ai-standard/nlp-1-7.png)

次に、各文章に対して、リストの単語があれば１、なければ０を入れます。この形式をBag of Wordsと呼びます。

![](https://s3.amazonaws.com/ai-standard/nlp-1-8.png)

以上で、テキストがベクトル形式になったので、これを特徴量として機械学習をしていくことが出来るようになりました。

次回は、もう一つのベクトル化手法であるword2vecというものについて勉強していきます。